In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Input, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

## Create batches for training, validation and testing

In [3]:
#find data sets. note these are not made public due to confidential nature.
train_path = "train_data/train"
valid_path = "train_data/valid"
test_path = "train_data/test"

In [4]:
#generate the batches that were also used for training.
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(240,240), classes=['Fissure', 'Racines', 'Normal'], batch_size=64)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(240,240), classes=['Fissure', 'Racines', 'Normal'], batch_size=64)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(240,240), classes=['Fissure', 'Racines', 'Normal'], batch_size=64, shuffle=False)

Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


## Build the model using weights

In [5]:
#the three following cells are used to recreate the model for which the weights were calculated
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [6]:
top_layers = efficientnet_model.output
top_layers = Flatten(name="flatten_top")(top_layers)
top_layers = Dense(1024, activation="ReLU",name="first_dense_top")(top_layers)
top_layers = Dropout(0.5, name="dropout_top")(top_layers)
top_layers = Dense(units=3, activation="softmax",name="linear_output")(top_layers)

In [7]:
model = Model(inputs=efficientnet_model.input, outputs=top_layers)

In [8]:
#loading the weights
cp_dir = "weights"
weights = tf.train.latest_checkpoint(cp_dir)
model.load_weights(weights)

AttributeError: 'NoneType' object has no attribute 'endswith'

In [9]:
#generating the predictions
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

ValueError: Asked to retrieve element 0, but the Sequence has length 0

## Printing the results

In [ ]:
#function for the confusion matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#plot the confusion matrix
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['Fissure','Racines', 'Normal']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
from IPython.display import Image, display

#calculate the accuracy of the model on the test set
y_true = test_batches.classes 
y_pred = np.argmax(predictions, axis=-1)
accuracy = (sum(y_true==y_pred)/len(y_true))
print("the accuracy is", accuracy)

#find the indices of the wrongly classified images
mistake = (y_pred!=y_true)
indices = tf.where(mistake)
proto_indices = tf.make_tensor_proto(indices)
indices = tf.make_ndarray(proto_indices)

#display the wrongly classified images
for ii in indices:
  if ii<200:
    image = os.listdir("Fissure")[ii[0]]
    display(Image("Fissure/"+image))

  elif ii<300:
    image = os.listdir("Racines")[ii[0]-200]
    display(Image("Racines/"+image))

  elif ii>=300:
    image = os.listdir("Normal")[ii[0]-300]
    display(Image("Normal/"+image))